In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1.

In [4]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [6]:
# df_q1.groupby("prod")["purchase"].sum()
df_q1.groupby("prod")["purchase"].sum().max()

27995166

In [9]:
ser_q1_agg = df_q1.groupby("prod")["purchase"].sum()
ser_q1_agg[ser_q1_agg == ser_q1_agg.max()]

prod
P00025442    27995166
Name: purchase, dtype: int64

In [10]:
top_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
top_prod

'P00025442'

In [11]:
df_q1_sub = df_q1.loc[df_q1["prod"] == top_prod, ]
df_q1_sub.head(2)

,prod,job,purchase
667,P00025442,17,19706
749,P00025442,7,15212


In [18]:
df_q1_sub.sort_values("purchase", ascending=False)

,prod,job,purchase
143485,P00025442,0,19707
534163,P00025442,12,19707
411315,P00025442,17,19706
259676,P00025442,0,19706
667,P00025442,17,19706
...,...,...,...
438091,P00025442,10,4186
237089,P00025442,2,4103
318228,P00025442,0,4091
346387,P00025442,1,4010


In [13]:
# df_q1_sub["job"].value_counts()
df_q1_sub["job"].value_counts().idxmax()

4

In [14]:
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [16]:
df_q1.sort_values(["prod", "purchase"], ascending = [False, False])

,prod,job,purchase
324997,P0099942,0,8833
542525,P0099942,20,7176
475839,P0099942,20,7120
379681,P0099942,4,7046
515435,P0099942,7,6928
...,...,...,...
382923,P00000142,1,2851
86632,P00000142,4,2833
242102,P00000142,4,2773
133259,P00000142,6,2754


### Q2.

In [33]:
df_q2_1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]].copy()
df_q2_1 = df_q2_1.fillna(0)
len(df_q2_1)

35

In [19]:
df.columns

Index(['user', 'prod', 'gender', 'age_group', 'job', 'city', 'marital',
       'prod_cat1', 'prod_cat2', 'prod_cat3', 'purchase'],
      dtype='object')

In [27]:
# df_q2_1.drop_duplicates()
df_q2_1.drop_duplicates().shape[0]

21

In [34]:
# df_q2_1["prod_cat"] = df_q2_1["prod_cat1"] + "-" + df_q2_1["prod_cat2"] + "-" + df_q2_1["prod_cat3"] 
df_q2_1["prod_cat1"] = df_q2_1["prod_cat1"].astype("int").astype("str")
df_q2_1["prod_cat2"] = df_q2_1["prod_cat2"].astype("int").astype("str")
df_q2_1["prod_cat3"] = df_q2_1["prod_cat3"].astype("int").astype("str")
df_q2_1["prod_cat"] = df_q2_1["prod_cat1"] + "-" + df_q2_1["prod_cat2"] + "-" + df_q2_1["prod_cat3"] 
df_q2_1.head(2)

,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,0,3-0-0
1,1,6,14,1-6-14


In [35]:
df_q2_1["prod_cat"].nunique()

21

In [36]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [39]:
target_cols = ["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]
df_q2 = df.loc[df["age_group"] == "26-35", target_cols].reset_index(drop = True)
df_q2 = df_q2.fillna(0)
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3
0,3,0,1,2.0,0.0
1,5,1,8,0.0,0.0


In [40]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")

In [42]:
df_q2["prod_cat"] = df_q2.loc[:, "prod_cat1":"prod_cat3"].apply(lambda x: x.str.cat(sep = "-"), axis = 1)
# .apply(lambda x: x.str.cat(sep = "-"), axis = 1)
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0


In [46]:
# df_q2.groupby("user")["prod_cat"].nunique()
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique()
df_q2_agg = df_q2_agg.reset_index()
df_q2_agg.head()

,user,marital,prod_cat
0,3,0,18
1,5,1,43
2,8,1,32
3,9,0,31
4,11,0,34


In [49]:
df_q2["user"].nunique()

2053

In [50]:
len(df_q2[["user", "marital"]].drop_duplicates())

2053

In [51]:
df_q2_agg.groupby("marital")["prod_cat"].mean()

marital
0    41.663183
1    41.792336
Name: prod_cat, dtype: float64

In [53]:
df_q2_agg.groupby("marital")["prod_cat"].mean().diff().round(2)

marital
0     NaN
1    0.13
Name: prod_cat, dtype: float64

### Q3.

In [55]:
df_q3_user = df[["user", "gender", "age_group", "job", "city", "marital"]]
df_q3_user = df_q3_user.drop_duplicates().reset_index(drop = True)
df_q3_user.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0


In [58]:
df_q3_user["user"].nunique(), len(df_q3_user)

(5891, 5891)

In [60]:
df_q3_agg1 = df.groupby("user")["prod"].nunique().reset_index()
df_q3_agg2 = df.groupby("user")["purchase"].sum().reset_index()
df_q3_agg2.head(2)

,user,purchase
0,1,334093
1,2,810472


In [86]:
df_q3_join = df_q3_user.merge(df_q3_agg1, on = "user")
df_q3_join = df_q3_join.merge(df_q3_agg2, on = "user")
df_q3_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [87]:
pd.crosstab(df_q3_join["gender"], (df_q3_join["gender"] == "M") + 0)

gender,0,1
gender,,
F,1666,0
M,0,4225


In [88]:
df_q3_join["gender"] = (df_q3_join["gender"] == "M") + 0

In [89]:
df_q3_join["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [90]:
ser_age = pd.Series(df_q3_join["age_group"].unique())
ser_age = ser_age.sort_values()
ser_age.astype("category").cat.codes

0    0
6    1
2    2
5    3
3    4
4    5
1    6
dtype: int8

In [91]:
ser_age = pd.Series(df_q3_join["age_group"].unique())
ser_age = ser_age.sort_values().reset_index(drop = True)
ser_age = pd.Series(ser_age.index, index = ser_age)
ser_age

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [92]:
pd.crosstab(df_q3_join["age_group"], 
            df_q3_join["age_group"].replace(ser_age))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [93]:
df_q3_join["age_group"] = df_q3_join["age_group"].replace(ser_age)

In [94]:
pd.cut([5, 10, 14, 20], bins = [0, 10, 20, 30], labels = [0, 1, 2])

[0, 0, 1, 1]
Categories (3, int64): [0 < 1 < 2]

In [95]:
df_q3_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472


In [97]:
# df_q3_dum = pd.get_dummies(df_q3_join)
df_q3_dum = pd.get_dummies(df_q3_join, columns = ["job", "city"])
df_q3_dum.head(1)

,user,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,1,0,0,0,35,334093,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [98]:
df_q3_dum = df_q3_dum.drop(columns = "user")
df_q3_dum.head(1)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [99]:
arr_q3_dum_nor = MinMaxScaler().fit_transform(df_q3_dum)
arr_q3_dum_nor[:1, ]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [101]:
model = KMeans(n_clusters = 7, random_state = 123)
model.fit(arr_q3_dum_nor)

C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7, random_state=123)

In [102]:
round(silhouette_score(arr_q3_dum_nor, labels = model.labels_), 2)

0.18